<a href="https://colab.research.google.com/github/kellyegorman/UNet-DenseNet-Blog/blob/main/segmentation-epistroma-unet/visualize_validation_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/PytorchDigitalPathology/segmentation_epistroma_unet

Mounted at /gdrive
/gdrive/MyDrive/PytorchDigitalPathology/segmentation_epistroma_unet


In [ ]:
#v1
#26/10/2018

dataname="epistroma" #should match the value used to train the network, will be used to load the appropirate model
gpuid=0


patch_size=256 #should match the value used to train the network
batch_size=1 #nicer to have a single batch so that we can iterately view the output, while not consuming too much
edge_weight=1

In [ ]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 9.1 MB/s eta 0:00:00


In [ ]:
import random, sys
import cv2
import glob
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.ndimage
import skimage
import time

import tables
from skimage import io, morphology
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms

from unet import UNet #code borrowed from https://github.com/jvanvugt/pytorch-unet
import PIL

In [ ]:
print(torch.cuda.get_device_properties(gpuid))
torch.cuda.set_device(gpuid)
device = torch.device(f'cuda:{gpuid}' if torch.cuda.is_available() else 'cpu')

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=9ceb3069-6415-ed3e-1a21-da54a14ad30d, L2_cache_size=4MB)


In [ ]:
checkpoint = torch.load(f"{dataname}_unet_best_model.pth")

<ipython-input-6-fb403b1de4a6>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"{dataname}_unet_best_model.pth")


In [ ]:
#load the model, note that the paramters are coming from the checkpoint, since the architecture of the model needs to exactly match the weights saved
model = UNet(n_classes=checkpoint["n_classes"], in_channels=checkpoint["in_channels"], padding=checkpoint["padding"],depth=checkpoint["depth"],
             wf=checkpoint["wf"], up_mode=checkpoint["up_mode"], batch_norm=checkpoint["batch_norm"]).to(device)
print(f"total params: \t{sum([np.prod(p.size()) for p in model.parameters()])}")
model.load_state_dict(checkpoint["model_dict"])

total params: 	122466


<All keys matched successfully>

In [ ]:
#this defines our dataset class which will be used by the dataloader
class Dataset(object):
    def __init__(self, fname ,img_transform=None, mask_transform = None, edge_weight= False):
        #nothing special here, just internalizing the constructor parameters
        self.fname=fname
        self.edge_weight = edge_weight

        self.img_transform=img_transform
        self.mask_transform = mask_transform

        self.tables=tables.open_file(self.fname)
        self.numpixels=self.tables.root.numpixels[:]
        self.nitems=self.tables.root.img.shape[0]
        self.tables.close()

        self.img = None
        self.mask = None
    def __getitem__(self, index):
        #opening should be done in __init__ but seems to be
        #an issue with multithreading so doing here
        if(self.img is None): #open in thread
            self.tables=tables.open_file(self.fname)
            self.img=self.tables.root.img
            self.mask=self.tables.root.mask

        #get the requested image and mask from the pytable
        img = self.img[index,:,:,:]
        mask = self.mask[index,:,:]

        #the original Unet paper assignes increased weights to the edges of the annotated objects
        #their method is more sophistocated, but this one is faster, we simply dilate the mask and
        #highlight all the pixels which were "added"
        if(self.edge_weight):
            weight = scipy.ndimage.morphology.binary_dilation(mask==1, iterations =2) & ~mask
        else: #otherwise the edge weight is all ones and thus has no affect
            weight = np.ones(mask.shape,dtype=mask.dtype)

        mask = mask[:,:,None].repeat(3,axis=2) #in order to use the transformations given by torchvision
        weight = weight[:,:,None].repeat(3,axis=2) #inputs need to be 3D, so here we convert from 1d to 3d by repetition

        img_new = img
        mask_new = mask
        weight_new = weight

        seed = random.randrange(sys.maxsize) #get a random seed so that we can reproducibly do the transofrmations
        if self.img_transform is not None:
            random.seed(seed) # apply this seed to img transforms
            img_new = self.img_transform(img)

        if self.mask_transform is not None:
            random.seed(seed)
            mask_new = self.mask_transform(mask)
            mask_new = np.asarray(mask_new)[:,:,0].squeeze()

            random.seed(seed)
            weight_new = self.mask_transform(weight)
            weight_new = np.asarray(weight_new)[:,:,0].squeeze()

        return img_new, mask_new, weight_new
    def __len__(self):
        return self.nitems

In [ ]:
#note that since we need the transofrmations to be reproducible for both masks and images
#we do the spatial transformations first, and afterwards do any color augmentations

#in the case of using this for output generation, we want to use the original images since they will give a better sense of the exepected
#output when used on the rest of the dataset, as a result, we disable all unnecessary augmentation.
#the only component that remains here is the randomcrop, to ensure that regardless of the size of the image
#in the database, we extract an appropriately sized patch
img_transform = transforms.Compose([
     transforms.ToPILImage(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=(patch_size,patch_size),pad_if_needed=True), #these need to be in a reproducible order, first affine transforms and then color
    #transforms.RandomResizedCrop(size=patch_size),
    #transforms.RandomRotation(180),
    #transforms.ColorJitter(brightness=0, contrast=0, saturation=0, hue=.5),
    #transforms.RandomGrayscale(),
    transforms.ToTensor()
    ])


mask_transform = transforms.Compose([
    transforms.ToPILImage(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(size=(patch_size,patch_size),pad_if_needed=True), #these need to be in a reproducible order, first affine transforms and then color
    #transforms.RandomResizedCrop(size=patch_size,interpolation=PIL.Image.NEAREST),
    #transforms.RandomRotation(180),
    ])

phases=["val"]
dataset={}
dataLoader={}
for phase in phases:

    dataset[phase]=Dataset(f"./{dataname}_{phase}.pytable", img_transform=img_transform , mask_transform = mask_transform ,edge_weight=edge_weight)
    dataLoader[phase]=DataLoader(dataset[phase], batch_size=batch_size,
                                shuffle=True, num_workers=0, pin_memory=True) #,pin_memory=True)

In [ ]:
%matplotlib inline

#set the model to evaluation mode, since we're only generating output and not doing any back propogation
model.eval()
for ii , (X, y, y_weight) in enumerate(dataLoader["val"]):
    X = X.to(device)  # [NBATCH, 3, H, W]
    y = y.type('torch.LongTensor').to(device)  # [NBATCH, H, W] with class indices (0, 1)

    output = model(X)  # [NBATCH, 2, H, W]

    output=output.detach().squeeze().cpu().numpy() #get output and pull it to CPU
    output=np.moveaxis(output,0,-1)  #reshape moving last dimension

    fig, ax = plt.subplots(1,4, figsize=(10,4))  # 1 row, 2 columns

    ax[0].imshow(output[:,:,1])
    ax[1].imshow(np.argmax(output,axis=2))
    ax[2].imshow(y.detach().squeeze().cpu().numpy())
    ax[3].imshow(np.moveaxis(X.detach().squeeze().cpu().numpy(),0,-1))
    plt.show()

**CREATE UNET PPTX**

In [ ]:
pip install python-pptx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 18.1 MB/s eta 0:00:00


In [ ]:
from pptx import Presentation
from pptx.util import Inches

from datetime import datetime
from skimage import color
import glob
import cv2
import numpy as np
from io import BytesIO
from tqdm.autonotebook import tqdm
import PIL.Image as Image


<ipython-input-12-d16cf567aa77>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
# -- Set meta data which will appear on first slide
title = "Epi/stroma segmentation"
date = datetime.today()
author = "Kelly Gorman"
comments = "data and code taken from blog andrewjanowczyk.com "
pptxfname = "epistroma_results.pptx"

In [ ]:
mask_files=glob.glob('./data/masks/*.png')

In [ ]:
#create presentation
prs = Presentation()
prs.slide_width = Inches(10)
prs.slide_height = Inches(10)

In [ ]:
blank_slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(blank_slide_layout)

In [ ]:
#make first slide with our metadata
slide.placeholders[0].text = title

tf = slide.placeholders[1].text_frame
tf.text = f'Date: {date}\n'
tf.text += f"Author: {author}\n"
tf.text += f"Comments: {comments}\n"

# -

In [ ]:
#wrapper function to add an image as a byte stream to a slide
#note that this is in place of having to save output directly to disk, and can be used in dynamic settings as well
def addimagetoslide(slide,img,left,top, height, width, resize = .5):
    res = cv2.resize(img , None, fx=resize,fy=resize ,interpolation=cv2.INTER_CUBIC) #since the images are going to be small, we can resize them to prevent the final pptx file from being large for no reason
    image_stream = BytesIO()
    Image.fromarray(res).save(image_stream,format="PNG")

    pic = slide.shapes.add_picture(image_stream, left, top ,height,width)
    image_stream.close()

In [ ]:
#helper function to blend two images
def blend2Images(img, mask):
    if (img.ndim == 3):
        img = color.rgb2gray(img)
    if (mask.ndim == 3):
        mask = color.rgb2gray(mask)
    img = img[:, :, None] * 1.0  # can't use boolean
    mask = mask[:, :, None] * 1.0
    out = np.concatenate((mask, img, mask), 2) * 255
    return out.astype('uint8')


# +


In [ ]:
# +
from pptx import Presentation
from pptx.util import Inches

from datetime import datetime
from skimage import color
import glob
import cv2
import numpy as np
from io import BytesIO
from tqdm.autonotebook import tqdm
import PIL.Image as Image
import os

# Ensure correct base path
base_path = "/gdrive/MyDrive/PytorchDigitalPathology/segmentation_epistroma_unet/data"

# Check if running in Google Colab and mount Drive
if "google.colab" in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = "/content/drive/MyDrive/PytorchDigitalPathology/segmentation_epistroma_unet/data"

# +
# -- Set meta data which will appear on first slide
title = "Epi/stroma segmentation"
date = datetime.today()
author = "Kelly Gorman"
comments = "data and code taken from blog andrewjanowczyk.com "
pptxfname = "/content/drive/MyDrive/epistroma_results.pptx"

# Only process images that have masks
mask_files = glob.glob(f"{base_path}/masks/*.png")

# +
# Create presentation
prs = Presentation()
prs.slide_width = Inches(10)
prs.slide_height = Inches(10)

blank_slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(blank_slide_layout)

# Add metadata to first slide
slide.placeholders[0].text = title

tf = slide.placeholders[1].text_frame
tf.text = f'Date: {date}\n'
tf.text += f"Author: {author}\n"
tf.text += f"Comments: {comments}\n"

# -

# Wrapper function to add an image as a byte stream to a slide
def addimagetoslide(slide, img, left, top, height, width, resize=0.5):
    res = cv2.resize(img, None, fx=resize, fy=resize, interpolation=cv2.INTER_CUBIC)  # Resize images to reduce pptx size
    image_stream = BytesIO()
    Image.fromarray(res).save(image_stream, format="PNG")

    pic = slide.shapes.add_picture(image_stream, left, top, height, width)
    image_stream.close()


# Helper function to blend two images
def blend2Images(img, mask):
    if img.ndim == 3:
        img = color.rgb2gray(img)
    if mask.ndim == 3:
        mask = color.rgb2gray(mask)
    img = img[:, :, None] * 1.0  # Can't use boolean
    mask = mask[:, :, None] * 1.0
    out = np.concatenate((mask, img, mask), 2) * 255
    return out.astype('uint8')


# +
for mask_fname in tqdm(mask_files):
    # Add a new slide for this set of images
    blank_slide_layout = prs.slide_layouts[0]
    slide = prs.slides.add_slide(blank_slide_layout)

    # Compute associated filenames
    orig_fname = mask_fname.replace(f"{base_path}/masks", base_path).replace("_mask.png", ".tif")
    output_fname = mask_fname.replace(f"{base_path}/masks", f"{base_path}/output").replace("_mask.png", "_class.png")

    # Check if original file exists before proceeding
    if not os.path.exists(orig_fname):
        print(f"Warning: {orig_fname} not found. Skipping...")
        continue

    # ------- Load and add original image
    img = cv2.cvtColor(cv2.imread(orig_fname), cv2.COLOR_BGR2RGB)
    addimagetoslide(slide, img, Inches(0), Inches(0), Inches(5), Inches(5))

    # ------ Load and add mask
    mask = cv2.cvtColor(cv2.imread(mask_fname), cv2.COLOR_BGR2RGB)
    addimagetoslide(slide, mask, Inches(5), Inches(0), Inches(5), Inches(5))

    # ------ Load and add output
    if os.path.exists(output_fname):  # Ensure output file exists
        output = cv2.cvtColor(cv2.imread(output_fname), cv2.COLOR_BGR2RGB)
        addimagetoslide(slide, output, Inches(5), Inches(5), Inches(5), Inches(5))
    else:
        print(f"Warning: {output_fname} not found. Skipping output image.")

    # ------ Fuse - load and add to slide
    addimagetoslide(slide, blend2Images(output, mask), Inches(0), Inches(5), Inches(5), Inches(5))

    # ------ Add results text box
    txBox = slide.shapes.add_textbox(Inches(10), Inches(0), Inches(4), Inches(4))
    tf = txBox.text_frame
    tf.text = f"{orig_fname}\n"
    tf.text += f"Overall Pixel Agreement: {(output == mask).mean():.4f}\n"
    tf.text += f"True Positive Rate: {(mask[output > 0] > 0).sum() / (output > 0).sum():.4f}\n"
    tf.text += f"False Positive Rate: {(mask[output == 0] > 0).sum() / (output == 0).sum():.4f}\n"
    tf.text += f"True Negative Rate: {(mask[output == 0] == 0).sum() / (output == 0).sum():.4f}\n"
    tf.text += f"False Negative Rate: {(mask[output > 0] == 0).sum() / (output > 0).sum():.4f}\n"

# Save presentation
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

prs.save(pptxfname)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  0%|          | 0/42 [00:00<?, ?it/s]

Mounted at /content/drive


In [ ]:
import os
print("Images:", os.listdir('./data'))
print("Masks:", os.listdir('./data/masks'))
print("Outputs:", os.listdir('./data/output'))


Images: ['10258_00006.tif', '10277_00006.tif', '10262_00025.tif', '10269_00022.tif', '10276_00037.tif', '10260_00022.tif', '10261_00002.tif', '10256_00003.tif', '10275_00099.tif', '10264_00056.tif', '10274_00004.tif', '10273_00095.tif', '10259_00002.tif', '10257_00026.tif', '10254_00001.tif', '10282_00016.tif', '10301_00026.tif', '10292_00018.tif', '10288_00003.tif', '10279_00049.tif', '10302_00098.tif', '10293_00011.tif', '10291_00012.tif', '10285_00007.tif', '10278_00006.tif', '10299_00154.tif', '10286_00014.tif', '10295_00012.tif', '10304_00005.tif', '12752_00004.tif', '10303_00090.tif', '10307_00004.tif', '12819_00004.tif', '12820_00005.tif', '12811_00008.tif', '12818_00006.tif', '12626_00016.tif', '10308_00048.tif', '12749_00010.tif', '10306_00018.tif', '12900_00008.tif', '12906_00017.tif', '12821_00018.tif', '12882_00026.tif', '12881_00009.tif', '12875_00002.tif', '12822_00002.tif', '12867_00005.tif', '12880_00001.tif', '12901_00005.tif', '12826_00003.tif', '12884_00018.tif', '12